In [1]:
import torch
import numpy as np

In [14]:
x = torch.rand(2, 4)
print(x)
mask = torch.tensor([[True, False, False, True], [False, False, True, True,]])
print(mask)

y = [x[i][mask[i]] for i in range(2)]
print(y)

tensor([[0.3166, 0.1579, 0.8411, 0.9223],
        [0.5220, 0.7898, 0.4168, 0.6027]])
tensor([[ True, False, False,  True],
        [False, False,  True,  True]])
[tensor([0.3166, 0.9223]), tensor([0.4168, 0.6027])]
